# Import modules

In [10]:
import tensorflow as tf
from facelib.dev.layer import DenseL2
from facelib.dev.loss import ArcFace
from facelib.dev.model import mobile_face_net
import numpy as np


# Prepare model architecture

In [11]:
input_shape = (112,112,3)
num_classes = 500
num_features = 128
model = mobile_face_net(input_shape = input_shape)
x = model.output
x = tf.keras.layers.BatchNormalization(name='features')(x)
x = DenseL2(num_classes)(x)
model = tf.keras.models.Model(model.input, x)


# Compile model

In [12]:
af = ArcFace(m=0.4, scale=30)
losses = [af.loss]
optimizer = tf.keras.optimizers.Adam()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]
model.compile(optimizer=optimizer, loss=losses, metrics=metrics)


# Print model architecture

In [13]:
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 56, 56, 64)        1792      
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_34 (Batc (None, 56, 56, 64)        256       
_________________________________________________________________
depthwise_conv2d_17 (Depthwi (None, 56, 56, 64)        640       
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 56, 56, 64)        0         
_________________________________________________________________
batch_normalization_35 (Batc (None, 56, 56, 64)        256 

# Dummy training data/label generator (batched)

In [14]:
num_epochs = 2
num_data = 1000
batch_size = 64
steps_per_epoch = num_data // batch_size

def data_gen():
    for i in range(steps_per_epoch):
        shape = (batch_size,) + input_shape
        dummy_img = np.random.rand(*shape).astype('float32')
        yield dummy_img # shape: (64,112,112,3)

def label_gen():
    for i in range(steps_per_epoch):
        dummy_label = np.random.randint(
            0, num_classes,
            (batch_size,)
        ) # shape: (64)
        one_hotted_label = np.eye(num_classes, dtype='float32')[dummy_label]
        yield one_hotted_label # shape: (64,500)


# Convert to tfdata

In [15]:
dt_data = tf.data.Dataset.from_generator(
    data_gen,
    output_types = (tf.float32),
    output_shapes = (64, 112, 112, 3)
)
dt_data = dt_data.repeat(num_epochs)
dt_data = dt_data.prefetch(2)

dt_label = tf.data.Dataset.from_generator(
    label_gen,
    output_types = (tf.float32),
    output_shapes = (64, 500)
)
dt_label = dt_label.repeat(num_epochs)
dt_label = dt_label.prefetch(2)

# shape: ((64,112,112,3), (64,500)
dt_train = tf.data.Dataset.zip((dt_data, dt_label))


# Initiate training

In [16]:
model.fit(
    dt_train,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
)


Train for 15 steps
Epoch 1/2
15/15 [==============================] - 27s 2s/step - loss: 21.2626 - acc: 0.0010
Epoch 2/2
15/15 [==============================] - 25s 2s/step - loss: 21.1868 - acc: 0.0010
